<a href="https://colab.research.google.com/github/NeuromatchAcademy/course-content-dl/blob/W1D3_anoop_changes/tutorials/W1D3_MLP/student/W1D3_Tutorial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Neuromatch Academy: Week 1, Day 3, Tutorial 1
# Multi-layer Perceptrons

__Content creators:__ Arash Ash



__Content reviewers:__ Saeed Salehi, Felix Bartsch, Yu-Fang Yang. 

__Content editors:__ Gagana B, Spiros Chavlis.

__Production editors:__ Anoop Kulkarni, Spiros Chavlis.  

---
# Tutorial objectives
In this tutorial, we delve deeper by using one of the most famous deep learning models of all!

MLPs are arguably one of the most tractable models that we can use to study deep learning fundamentals. Here we will learn why MLPs are: 

* similar to biological networks
* good at function approximation

In [ ]:
#@markdown Tutorial slides
# you should link the slides for all tutorial videos here (we will store pdfs on osf)

from IPython.display import HTML
HTML('<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vSPvHqDTmMq4GyQy6lieNEFxq4qz1SmqC2RNoeei3_niECH53zneh8jJVYOnBIdk0Uaz7y2b9DK8V1t/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>')

# Recap the experience from last week

We focused on linear deep learning last week. We discussed Artificial Neural Networks and saw how it works, the dynamics of learning, and the properties of high dimensional spaces. You should now have some intuition about deep learning systems we will learn. We also dived into PyTorch and autograd, which are tools that make our life easy. 

In [ ]:
#@title Video 1: Linear DL
# Insert the ID of the corresponding youtube video
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="xlYttP5C_LY", width=854, height=480, fs=1)
print("Video available at https://youtu.be/" + video.id)
video

Meet with your pod for 10 minutes to discuss what you learned, what was clear, and what you hope to learn more about.

In [ ]:
#@markdown Tell us your thoughts about what you have learned in week 2.
w2_upshot = '' #@param {type:"string"}

# Question of the day

In [ ]:
#@markdown What functional forms are good or bad for representing complex functions?
w3_q = '' #@param {type:"string"}

---
# Setup

In [ ]:
#@title Imports
# imports
import random
import pathlib

import torch
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import make_grid
from IPython.display import HTML, display

# Make sure the Runtime is with None to ensure compatibility
dev = "cpu"

In [ ]:
# @title Seeding for reproducibility
seed = 522
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
def seed_worker(worker_id):
    worker_seed = seed % (worker_id+1)
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [ ]:
# @title Dataset download
%%capture
!rm -r AnimalFaces32x32/
!git clone https://github.com/arashash/AnimalFaces32x32
!rm -r afhq/
!unzip ./AnimalFaces32x32/afhq_32x32.zip

In [ ]:
# @title Figure settings
import ipywidgets as widgets
%matplotlib inline
fig_w, fig_h = (8, 6)
plt.rcParams.update({'figure.figsize': (fig_w, fig_h)})
%config InlineBackend.figure_format = 'retina'
my_layout = widgets.Layout()

In [ ]:
# @title Helper functions
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis(False)
    plt.show()

def progress(epoch, loss, epochs=100):
    return HTML("""
        <label for="file">Training loss: {loss}</label>
        <progress
            value='{epoch}'
            max='{epochs}',
            style='width: 100%'
        >
            {epoch}
        </progress>
    """.format(loss=loss, epoch=epoch, epochs=epochs))

def plot_function_approximation(x, relu_acts, y_hat):
  fig, axes = plt.subplots(2, 1)

  # Plot ReLU Activations
  axes[0].plot(x, relu_acts.T);
  axes[0].set(xlabel = 'x', ylabel = 'Activation', title = 'ReLU Activations')
  labels = [f'ReLU {i + 1}' for i in range(relu_acts.shape[0])]
  axes[0].legend(labels, ncol = 2)

  # Plot function approximation
  axes[1].plot(x, torch.sin(x), label = 'truth')
  axes[1].plot(x, y_hat, label = 'estimated')
  axes[1].legend()
  axes[1].set(xlabel = 'x', ylabel = 'y(x)', title = 'Function Approximation');

  plt.tight_layout()
  plt.show()

---
# Section 1: Neuron Physiology

In [ ]:
#@title Video 1.1: Overview and Integrate-and-Fire Neurons
# Insert the ID of the corresponding youtube video
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="exTzHGfEAvU", width=854, height=480, fs=1)
print("Video available at https://youtu.be/" + video.id)
video

## Section 1.1: Leaky Integrate-and-fire (LIF)
The basic idea of LIF neuron was proposed in 1907 by Louis Édouard Lapicque, long before we understood the electrophysiology of a neuron (see a translation of [Lapicque's paper](https://pubmed.ncbi.nlm.nih.gov/17968583/) ). More details of the model can be found in the book [**Theoretical neuroscience**](http://www.gatsby.ucl.ac.uk/~dayan/book/) by Peter Dayan and Laurence F. Abbott.

The model dynamics is defined with the following formula,

$$
\frac{d V}{d t}=\left\{\begin{array}{cc}
\frac{1}{C}\left(-\frac{V}{R}+I \right) & t>t_{r e s t} \\
0 & \text { otherwise }
\end{array}\right.
$$

Note that $V$, $C$, and $R$ are the membrane voltage, capacitance, and resitance of the neuron respectively and $-\frac{V}{R}$ is the leakage current. When $I$ is sufficiently strong such that $V$ reaches a certain threshold value $V_{\rm th}$, it momentarily spikes and then $V$ is reset to $V_{\rm reset}< V_{\rm th}$, and voltage stays at $V_{\rm reset}$ for $\tau_{\rm ref}$ ms, mimicking the refractoriness of the neuron during an action potential (note that $V_{\rm reset}$ and $\tau_{\rm ref}$ is assumed to be zero in the lecture):

\begin{eqnarray}
V(t)=V_{\rm reset} \text{  for } t\in(t_{\text{sp}}, t_{\text{sp}} + \tau_{\text{ref}}]
\end{eqnarray}

where $t_{\rm sp}$ is the spike time when $V(t)$ just exceeded $V_{\rm th}$.

Thus, the LIF model captures the facts that a neuron:
- performs spatial and temporal integration of synaptic inputs 
- generates a spike when the voltage reaches a certain threshold
- goes refractory during the action potential
- has a leaky membrane 

For in-depth content on computational models of neurons, follow the [NMA](https://www.neuromatchacademy.org/) Week 3 Day 1 material on Real Neurons and specifically this [Tutorial](https://colab.research.google.com/github/NeuromatchAcademy/course-content/blob/master/tutorials/W3D1_RealNeurons/W3D1_Tutorial1.ipynb).


## Section 1.2: Simulating an LIF Neuron

In the cell below is given a function for LIF neuron model with it's arguments described.

Note that we will use Euler's method to make a numerical approximation to a derivative. Hence we will use the following implementation of the model dynamics,

$$
V_n=\left\{\begin{array}{cc}
V_{n-1} + \frac{1}{C}\left(-\frac{V}{R}+I \right) \Delta t & t>t_{r e s t} \\
0 & \text { otherwise }
\end{array}\right.
$$

In [ ]:
def run_LIF(I, T = 50, dt = 0.1, t_rest = 0, tau_ref = 4,
            Rm = 1, Cm = 10, Vth = 1, V_spike = 0.5):
  """
  Simulate the LIF dynamics with external input current

  Args:
    I          : input current (mA)
    T          : total time to simulate (msec)
    dt         : simulation time step (msec)
    t_rest     : initial refractory time
    tau_ref    : refractory period (msec)
    Rm         : resistance (kOhm)
    Cm         : capacitance (uF)
    Vth        : spike threshold (V)
    V_spike    : spike delta (V)

  Returns:
    time       : time points
    Vm         : membrane potentials
  """

  # Set up array of time steps
  time = torch.arange(0, T+dt, dt)

  # Set up array for tracking Vm
  Vm = torch.zeros(len(time))

  # Iterate over each time step
  for i, t in enumerate(time):

    # If t is after refractory period
    if t > t_rest:
      Vm[i] = Vm[i-1] + 1/Cm*(-Vm[i-1]/Rm + I)  * dt

    # If Vm is over the threshold
    if Vm[i] >= Vth:

      # Increase volatage by change due to spike
      Vm[i] += V_spike

      # Set up new refactory period
      t_rest = t + tau_ref

  return time, Vm


### Uncomment below to test your function
sim_time, Vm = run_LIF(1.5)
with plt.xkcd():
  plt.plot(sim_time, Vm)
  plt.title('LIF Neuron Output')
  plt.ylabel('Membrane Potential (V)')
  plt.xlabel('Time (msec)')
  plt.show()

## Section 1.3: Nonlinearity of LIF neurons

In [ ]:
#@title Video 1.3: Are Integrate-and-Fire Neurons Linear?
# Insert the ID of the corresponding youtube video
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="6IzHZB7xf34", width=854, height=480, fs=1)
print("Video available at https://youtu.be/" + video.id)
video

## Interactive demo: F-I explorer for different $R_m$
We know that neurons communicate by modulating the spike count. Therefore it makes sense to characterize their spike count as a function of input current. This is called the neuron's input-output transfer function (so simply F-I curve). Let's plot the neuron's F-I curve and see how it changes with respect to the membrane resistance? 

In [ ]:
# @title

# @markdown Make sure you execute this cell to enable the widget!

@widgets.interact(Rm=widgets.FloatSlider(1., min=0.5, max=10., step=0.1, layout=my_layout))

def plot_IF_curve(Rm):
  T = 100 # total time to simulate (msec)
  dt = 1 # simulation time step (msec)
  Vth = 1 # spike threshold (V)
  Is = torch.linspace(0, 2, 10)
  spike_counts = []
  for I in Is:
    _, Vm = run_LIF(I, T = T, Vth = Vth, Rm=Rm)
    spike_counts += [torch.sum(Vm > Vth)]

  plt.plot(Is, spike_counts)
  plt.title('LIF Transfer Function (I/F Curve)')
  plt.ylabel('Spike count')
  plt.xlabel('I (mA)')
  plt.show()

In [ ]:
#@markdown What happens at infinite membrane potential (Rm)? Why?
w3_why_become_relu = '' #@param {type:"string"}

---
# Section 2: The Need for MLPs

In [ ]:
#@title Video 2: The XOR problem
# Insert the ID of the corresponding youtube video
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="PERmPT1cOP0", width=854, height=480, fs=1)
print("Video available at https://youtu.be/" + video.id)
video

---
## Section 2.1: Universal Function Approximation Theorem

In [ ]:
#@title Video 2.1: Universal Approximation
# Insert the ID of the corresponding youtube video
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="XXXYxolMVdw", width=854, height=480, fs=1)
print("Video available at https://youtu.be/" + video.id)
video

Check out [Lipschitz continuity](https://en.wikipedia.org/wiki/Lipschitz_continuity) for more information about the slide.

Now the question is can we approximate any function using multilayer perceptrons?

 Universal Approximation theorem proves that we can! The intuition behind the theorem is that we can approximate any function sufficiently well, given a sufficient number of basis functions. These basis functions are present in the hidden layer of MLPs.  

## Exercise 1: Function approximation with ReLU
We learned that one hidden layer MLPs are enough to approximate any smooth function! Now let's manually fit a Sine function using ReLU activation. 


We will approximate the sine function using a linear combination (a weighted sum) of ReLUs with slope 1. We need to determine the bias terms (which determines where the ReLU inflection point from 0 to linear occurs) and how to weight each ReLU. The idea is to set the weights iteratively so that the slope changes in the new sample's direction.

First, we generate our "training data" from a sine function. These are the points we will use to learn how to approximate the function. We have 10 training data points so we will have 9 ReLUs (we don't need a ReLU for the last data point as we don't have anything to the right of it to model). 

We first need to figure out the bias term for each ReLU and compute the activation of each ReLU where:

$$ y(x) = max(0, x + b) $$

We then need to figure out the correct weights on each ReLU so the linear combination approximates the desired function.

In [ ]:
def approximate_function(x_train, y_train):

    ####################################################################
    # Fill in missing code below (...),
    # then remove or comment the line below to test your function
    raise NotImplementedError("Complete approximate_function!")
    ####################################################################

    # Number of relus
    n_relus = x_train.shape[0] - 1

    # x axis points (more than x train)
    x = torch.linspace(torch.min(x_train), torch.max(x_train), 1000)

    ## COMPUTE RELU ACTIVATIONS

    # First determine what bias terms should be for each of 9 ReLUs
    b = ...

    # Compute ReLU activations for each point along the x axis (x)
    relu_acts = torch.zeros((n_relus, x.shape[0]))

    for i_relu in range(n_relus):
      relu_acts[i_relu, :] = torch.relu(x + b[i_relu])

    ## COMBINE RELU ACTIVATIONS

    # Set up weights for weighted sum of ReLUs
    combination_weights = torch.zeros((n_relus, ))

    # Figure out weights on each ReLU
    prev_slope = 0
    for i in range(n_relus):
      delta_x = x_train[i+1] - x_train[i]
      slope = (y_train[i+1] - y_train[i]) / delta_x
      combination_weights[i] = ...
      prev_slope = slope

    # Get output of weighted sum of ReLU activations for every point along x axis
    y_hat = ...

    return y_hat, relu_acts, x

# Make training data from sine function
N_train = 10
x_train = torch.linspace(0, 2*np.pi, N_train).view(-1, 1)
y_train = torch.sin(x_train)


### uncomment the lines below to test your function approximation
# y_hat, relu_acts, x = approximate_function(x_train, y_train)
# plot_function_approximation(x, relu_acts, y_hat)

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content-dl/tree/main//tutorials/W1D3_MLP/solutions/W1D3_Tutorial1_Solution_466c7b35.py)

*Example output:*

<img alt='Solution hint' align='left' width=546 height=415 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content-dl/main/tutorials/W1D3_MLP/static/W1D3_Tutorial1_Solution_466c7b35_0.png>



---
# Section 3: MLPs in Pytorch

In [ ]:
#@title Video 3: Making Multi-Layer Perceptrons
# Insert the ID of the corresponding youtube video
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="bAhrg8Z8_r8", width=854, height=480, fs=1)
print("Video available at https://youtu.be/" + video.id)
video

In the previous segment, we implemented a function to approximate any smooth function using MLPs. We saw that using Lipschitz continuity; we can prove that our approximation is mathematically correct. MLPs are fascinating, but before we get into the details on designing them, let's familiarize ourselves with some basic terminology of MLPs- layer, neuron, depth, width, weight, bias, and activation function. Armed with these ideas, we can now design an MLP given its input, hidden layers, and output size.

## Exercise 2: Implement a general-purpose MLP in Pytorch
The objective is to design an MLP with these properties:
* works with any input (1D, 2D, etc.)
* construct any number of given hidden layers using ModuleList
* use the same given activation function in all hidden layers

In [ ]:
class Net(nn.Module):
    def __init__(self, actv, num_inputs, hidden_units, num_outputs):
        super(Net, self).__init__()

        # Assign activation function (exec allows us to assign function from string)
        exec('self.actv = nn.%s'%actv)

        # Initialize layers of MLP
        self.layers = nn.ModuleList()

        ####################################################################
        # Fill in missing code below (...),
        # then remove or comment the line below to test your function
        raise NotImplementedError("Create MLP Layers")
        ####################################################################

        # Loop over layers and create each one
        for i in range(len(hidden_units)):
          next_num_inputs = hidden_units[i]
          self.layers += ...
          num_inputs = next_num_inputs

        # Create final layer
        self.out = nn.Linear(num_inputs, num_outputs)

    def forward(self, x):

        ####################################################################
        # Fill in missing code below (...),
        # then remove or comment the line below to test your function
        raise NotImplementedError("Calculate the forward pass")
        ####################################################################

        # Flatten inputs to 2D (if more than that)
        x = ...

        # Get activations of each layer
        for layer in self.layers:
          x = ...

        # Get outputs
        x = self.out(x)

        return x

### Uncomment below to create network and test it on input
# net = Net(actv='LeakyReLU(0.1)',
#     num_inputs = 2,
#     hidden_units = [100, 10, 5],
#     num_outputs = 1)

# input = torch.zeros((100, 2))
# y = net(input)
# print(f'The output shape is {y.shape} for an input of shape {input.shape}')

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content-dl/tree/main//tutorials/W1D3_MLP/solutions/W1D3_Tutorial1_Solution_4fa68815.py)



## Section 3.1: Classification with MLPs

In [ ]:
#@title Video 3.1: Classification and CrossEntropy
# Insert the ID of the corresponding youtube video
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="bxzDe2pifKU", width=854, height=480, fs=1)
print("Video available at https://youtu.be/" + video.id)
video

Two potential loss functions we could use out of the box for multi-class classification are:
* CrossEntropyLoss:
This criterion expects a class index in the range $[0, C-1]$ as the target (labels) for each value of a $1D$ tensor of size minibatch (`N`). There are other optional parameters like class weights and class ignores. Check the documentation here for more detail. 

To get CrossEntropyLoss of a sample $i$, we could first calculate $-\log(\text{softmax(x}))$ and then take the element corresponding to $\text { label }_i$ as the loss. However, due to numerical stability, we implement this more stable equivalent form,

$$
\operatorname{loss}(x_i, \text { label }_i)=-\log \left(\frac{\exp (x[\text { label }_i])}{\sum_{j} \exp (x[j])}\right)=-x_i[\text { label }_i]+\log \left(\sum_{j=1}^C \exp (x_i[j])\right)
$$

* MultiMarginLoss: as a bonus you could see bellow cell for implementation

## Bonus: Multi Margin Loss

The loss corresponding to class j is calculated as follows,
$$
l_j(x, label)=\sum_{j\neq label} \max (0, \operatorname{margin}-x[label]+x[j])
$$

## Exercise 3: Implement Batch Cross Entropy Loss

Since we will be doing batch learning, we'd like a loss function that given:
* a batch of predidictions `x` with shape `(N, C)` 
* a batch of `labels` with shape `(N, )` that ranges from `0` to `C-1`

returns the average loss $L$ calculated according to:
$$
loss(x_i, \text { label }_i)=-x_i[\text { label }_i]+\log \left(\sum_{j=1}^C \exp (x_i[j])\right)
$$

$$
L = \frac{1}{N} \sum_{i=1}^{N}{loss(x_i, \text { label }_i)}
$$

Steps:

1.   Use indexing operation to get predictions of class corresponding to the labels (i.e., $x_i[\text { label }_i]$)
2.   Compute $loss(x_i, \text { label }_i)$ vector (`losses`) using `torch.log()` and `torch.exp()` without Loops!
3. Return the average of the loss vector



In [ ]:
def cross_entropy_loss(x, labels):
  ####################################################################
  # Fill in missing code below (...),
  # then remove or comment the line below to test your function
  raise NotImplementedError("Cross Entropy Loss")
  ####################################################################

  x_of_labels = torch.zeros(len(labels))
  for i, label in enumerate(labels):
    # 1. prediction for each class corresponding to the label
    x_of_labels[i] = ...

  # 2. loss vector for the batch
  losses = ...

  # 3. Return the average of the loss vector
  avg_loss = ...

  return avg_loss

### Uncomment below to test your function
# labels = torch.tensor([0,
#                        1])
# x = torch.tensor([[10.0, 1.0, -1.0, -20.0], # correctly classified
#                   [10.0, 10.0, 85.0, -110.0]]) # Not correctly classified

# our_loss = cross_entropy_loss(x, labels).item()

# CE = nn.CrossEntropyLoss()
# pytorch_loss = CE(x, labels).item()
# print('Our CE loss: %0.8f, Pytorch CE loss: %0.8f'%(our_loss, pytorch_loss))

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content-dl/tree/main//tutorials/W1D3_MLP/solutions/W1D3_Tutorial1_Solution_1090fadb.py)



## Section 3.2: Spiral classification dataset
Before we could start optimizing these loss functions, we need a dataset!

Let's turn this fancy-looking equation into a classification dataset

$$
\begin{array}{c}
X_{k}(t)=t\left(\begin{array}{c}
\sin \left[\frac{2 \pi}{K}\left(2 t+k-1\right)\right]+\mathcal{N}\left(0, \sigma\right) \\
\cos \left[\frac{2 \pi}{K}\left(2 t+k-1\right)\right]+\mathcal{N}\left(0, \sigma\right) 
\end{array}\right)
\end{array}, \quad 0 \leq t \leq 1, \quad k=1, \ldots, K
$$

In [ ]:
def create_spiral_dataset(K, sigma, N):

    # Initialize t, X, y
    t = torch.linspace(0, 1, N)
    X = torch.zeros(K*N, 2)
    y = torch.zeros(K*N)

    # Create data
    for k in range(K):
      X[k*N:(k+1)*N, 0] = t*(torch.sin(2*np.pi/K*(2*t+k)) + sigma*torch.randn(N))
      X[k*N:(k+1)*N, 1] = t*(torch.cos(2*np.pi/K*(2*t+k)) + sigma*torch.randn(N))
      y[k*N:(k+1)*N] = k

    return X, y

# Set parameters
K = 4
sigma = 0.16
N = 1000

### Uncomment below to visualize data when done
X, y = create_spiral_dataset(K, sigma, N)
with plt.xkcd():
  plt.scatter(X[:, 0], X[:, 1], c = y)
  plt.show()

## Section 3.3: Training and Evaluation

In [ ]:
#@title Video 3.3: Cross-Validation
# Insert the ID of the corresponding youtube video
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="U6BFrVCLsWU", width=854, height=480, fs=1)
print("Video available at https://youtu.be/" + video.id)
video

## Exercise 4: Implement it for a classfication task
Now that we have the Spiral dataset and loss function, it's your turn to implement a simple train/test split for training and validation.

Steps to follow: 
  * Dataset shuffle
  * Train/Test split
  * Dataloader definition
  * Training and Evaluation

In [ ]:
def shuffle_and_split_data(X, y):

  ####################################################################
  # Fill in missing code below (...),
  # then remove or comment the line below to test your function
  raise NotImplementedError("Shuffle & split data")
  ####################################################################

  # Number of samples
  N = X.shape[0]

  # Shuffle data
  shuffled_indices = ...   # get indices to shuffle data
  X = X[shuffled_indices]
  y = y[shuffled_indices]

  # Split data into train/test
  test_size = ...    # assign size of test data
  X_test = X[:test_size]
  y_test = y[:test_size]
  X_train = X[test_size:]
  y_train = y[test_size:]

  return X_test, y_test, X_train, y_train


### Uncomment below to test your function
#X_test, y_test, X_train, y_train = shuffle_and_split_data(X, y)
#with plt.xkcd():
#  plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test)
#  plt.title('Test data')
#  plt.show()

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content-dl/tree/main//tutorials/W1D3_MLP/solutions/W1D3_Tutorial1_Solution_ddc9124a.py)

*Example output:*

<img alt='Solution hint' align='left' width=497 height=378 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content-dl/main/tutorials/W1D3_MLP/static/W1D3_Tutorial1_Solution_ddc9124a_0.png>



And we need to make a Pytorch data loader out of it. Data loading in PyTorch can be separated in 2 parts:
* Data must be wrapped on a Dataset parent class where the methods __getitem__ and __len__ must be overrided. Not that at this point the data is not loaded on memory. PyTorch will only load what is needed to the memory. Here `TensorDataset` does this for us directly.
* Use a Dataloader that will actually read the data in batches and put into memory. Also, the option of `num_workers > 0` allows multithreading, which prepares multiple batches in the queue to speed things up.

In [ ]:
batch_size = 128
test_data = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_data, batch_size=batch_size,
                         shuffle=False, num_workers=0)

train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, drop_last=True,
                        shuffle=True, num_workers=0, worker_init_fn=seed_worker)

Let's write a general-purpose training and evaluation code and keep it in our pocket for next tutorial as well. So make sure you review it to see what it does.

Note that `model.train()` tells your model that you are training the model. So effectively layers like dropout, batchnorm etc. which behave different on the train and test procedures know what is going on and hence can behave accordingly. And to turn off training mode we set `model.eval()`

In [ ]:
def train_test_classification(net, criterion, optimizer,
                              train_loader, test_loader,
                              num_epochs=1, verbose=True,
                              training_plot=False):
  if verbose:
    progress_bar = display(progress(0, 0, num_epochs), display_id=True)

  net.train()
  training_losses = []
  for epoch in range(num_epochs):  # loop over the dataset multiple times
      running_loss = 0.0
      for i, data in enumerate(train_loader, 0):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data
          inputs = inputs.to(dev).float()
          labels = labels.to(dev).long()

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = net(inputs)

          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          if verbose:
            training_losses += [loss.item()]
            running_loss += loss.item()
            if i % 10 == 9:    # update every 10 mini-batches
                progress_bar.update(progress(epoch+1, running_loss / 10, num_epochs))
                running_loss = 0.0

  net.eval()
  def test(data_loader):
    correct = 0
    total = 0
    for data in data_loader:
        inputs, labels = data
        inputs = inputs.to(dev).float()
        labels = labels.to(dev).long()

        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    return total, acc

  train_total, train_acc = test(train_loader)
  test_total, test_acc = test(test_loader)

  if verbose:
    print('Accuracy on the %d training samples: %0.2f %%' % (train_total, train_acc))
    print('Accuracy on the %d testing samples: %0.2f %%' % (test_total, test_acc))

  if training_plot:
    plt.plot(training_losses)
    plt.xlabel('Batch')
    plt.ylabel('Training loss')
    plt.show()

  return train_acc, test_acc

In [ ]:
#@markdown Is it necessary to use `net.train()` and `net.eval()` for our MLP model? why?
w3_why_two_modes = '' #@param {type:"string"}

Now let's put everything together and train your first deep-ish model!

In [ ]:
net = Net('ReLU()', X_train.shape[1], [128], K).to(dev)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)
num_epochs = 100
_, _ = train_test_classification(net, criterion, optimizer, train_loader,
                                 test_loader, num_epochs=num_epochs,
                                 training_plot=True)

And finally, let's visualize the learned decision-map. We know you're probably running out of time, so we won't make you write code now! But make sure you have reviewed it since we'll start with another visualization technique next time.

In [ ]:
def sample_grid(M=500, x_max = 2.0):
  ii, jj = torch.meshgrid(torch.linspace(-x_max, x_max,M),
                          torch.linspace(-x_max, x_max, M))
  X_all = torch.cat([ii.unsqueeze(-1),
                     jj.unsqueeze(-1)],
                     dim=-1).view(-1, 2)
  return X_all

def plot_decision_map(X_all, y_pred, X_test, y_test, M=500, x_max = 2.0, eps = 1e-3):
  decision_map = torch.argmax(y_pred, dim=1)

  for i in range(len(X_test)):
    indeces = (X_all[:, 0] - X_test[i, 0])**2 + (X_all[:, 1] - X_test[i, 1])**2 < eps
    decision_map[indeces] = (K + y_test[i]).long()

  decision_map = decision_map.view(M, M).cpu()
  plt.imshow(decision_map, extent=[-x_max, x_max, -x_max, x_max], cmap='jet')
  plt.plot()

X_all = sample_grid()
y_pred = net(X_all)
plot_decision_map(X_all, y_pred, X_test, y_test)

In [ ]:
#@markdown Do you think this model is performing well outside its training distribution? Why?
w3_OoD = '' #@param {type:"string"}

# Submit your responses
Please run the following cell and then press "Submit" so we can record your responses.

In [ ]:
import time
import numpy as np
from IPython.display import IFrame
#@markdown #Run Cell to Show Airtable Form
#@markdown ##**Confirm your answers and then click "Submit"**

def prefill_form(src, fields: dict):
  '''
  src: the original src url to embed the form
  fields: a dictionary of field:value pairs,
  e.g. {"pennkey": my_pennkey, "location": my_location}
  '''
  prefills = "&".join(["prefill_%s=%s"%(key, fields[key]) for key in fields])
  src = src + prefills
  src = "+".join(src.split(" "))
  return src


#autofill time if it is not present
try: t0;
except NameError: t0 = time.time()
try: t1;
except NameError: t1 = time.time()
try: t2;
except NameError: t2 = time.time()
try: t3;
except NameError: t3 = time.time()

#autofill fields if they are not present
#a missing pennkey and pod will result in an Airtable warning
#which is easily fixed user-side.
try: my_pennkey;
except NameError: my_pennkey = ""

try: my_pod;
except NameError: my_pod = "Select"

try: w2_upshot;
except NameError: w2_upshot = ""

try: w3_q;
except NameError: w3_q = ""

try: w3_min_xor;
except NameError: w3_min_xor = ""

try: w3_why_become_relu;
except NameError: w3_why_become_relu = ""

try: w3_why_two_modes;
except NameError: w3_why_two_modes = ""

try: w3_OoD;
except NameError: w3_OoD = ""


times = np.array([t1,t2,t3])-t0

fields = {"pennkey": my_pennkey,
          "pod": my_pod,
          "w2_upshot":w2_upshot,
          "w3_q": w3_q,
          "w3_why_become_relu":w3_why_become_relu,
          "w3_min_xor": w3_min_xor,
          "w3_why_two_modes":w3_why_two_modes,
          "w3_OoD":w3_OoD,
          "cumulative_times": times}

src = "https://airtable.com/embed/shrO0aY7Sz8u8qY2H?"

#now instead of the original source url, we do: src = prefill_form(src, fields)
display(IFrame(src = prefill_form(src, fields), width = 800, height = 400))

## Feedback
How could this session have been better? How happy are you in your group? How do you feel right now?

Feel free to use the embeded form below or use this link:
<a target="_blank" rel="noopener noreferrer" href="https://airtable.com/shrNSJ5ECXhNhsYss">https://airtable.com/shrNSJ5ECXhNhsYss</a>

In [ ]:
# @title Feedback form
display(IFrame(src="https://airtable.com/embed/shrNSJ5ECXhNhsYss?backgroundColor=red", width = 800, height = 400))